# Unsloth Fine-tuning on Multiple Datasets (Quick Version)

This notebook demonstrates how to use Unsloth to fine-tune a language model on a small subset of multiple datasets (SlimOrca and Capybara) using PEFT.

In [1]:
# Install necessary dependencies for Unsloth
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

# Update transformers
!pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"

# Uninstall xformers and reinstall the nightly version
!pip uninstall xformers -y
!pip install xformers --pre -f https://download.pytorch.org/whl/nightly/cu121/torch_nightly.html

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-pxknn979/unsloth_4ca7c1a20308439595fb903c05b8051f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-pxknn979/unsloth_4ca7c1a20308439595fb903c05b8051f
  Resolved https://github.com/unslothai/unsloth.git to commit 3085f4c3daacc63939e78e3c87759d0d03c5a71f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━

In [2]:
# Import necessary libraries
import bitsandbytes as bnb
from datasets import load_dataset, Dataset, concatenate_datasets
from unsloth import FastLanguageModel
from transformers import TrainingArguments, AutoTokenizer
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
# Load a small subset of datasets
slimorca = load_dataset("Open-Orca/SlimOrca", split="train[:1000]")
capybara = load_dataset("LDJnr/Capybara", split="train[:1000]")

print(f"SlimOrca sample size: {len(slimorca)}")
print(f"Capybara sample size: {len(capybara)}")

# Inspect the structure of both datasets
print("\nSlimOrca sample:")
print(slimorca[0])
print("\nCapybara sample:")
print(capybara[0])

README.md:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

oo-labeled_correct.gpt4.sharegpt.jsonl:   0%|          | 0.00/986M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/517982 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/6.47k [00:00<?, ?B/s]

CapybaraPure_Decontaminated.jsonl:   0%|          | 0.00/74.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16006 [00:00<?, ? examples/s]

SlimOrca sample size: 1000
Capybara sample size: 1000

SlimOrca sample:
{'conversations': [{'from': 'system', 'value': 'You are an AI assistant. You will be given a task. You must generate a detailed and long answer.', 'weight': None}, {'from': 'human', 'value': 'Write an article based on this "A man has been charged with murder and attempted murder after a woman and the man she was on a date with were stabbed at a restaurant in Sydney, Australia."', 'weight': 0.0}, {'from': 'gpt', 'value': "Title: Tragedy Strikes in Sydney: Victims Stabbed during Dinner Date, Suspect Charged with Murder and Attempted Murder\n\nIntroduction\n\nA seemingly ordinary evening took a tragic turn in Sydney, Australia when a romantic dinner date at a local restaurant ended in bloodshed, as a man allegedly attacked a couple, stabbing both victims. Local authorities have charged the suspect with murder and attempted murder in connection with the horrific crime. In this article, we delve into the chilling incide

In [4]:
def clean_slimorca(example):
    conversations = example["conversations"]
    text = ""
    for turn in conversations:
        if turn["from"] == "human":
            text += f"Human: {turn['value']}\n"
        elif turn["from"] == "gpt":
            text += f"Assistant: {turn['value']}\n"
    return {"text": text.strip()}

def clean_capybara(example):
    if isinstance(example["conversation"], list):
        text = ""
        for turn in example["conversation"]:
            text += f"Human: {turn['input']}\nAssistant: {turn['output']}\n"
    else:
        text = f"Human: {example['source']}\nAssistant: {example['conversation']}"
    return {"text": text.strip()}

# Clean datasets
slimorca_cleaned = slimorca.map(clean_slimorca)
capybara_cleaned = capybara.map(clean_capybara)

# Combine datasets
combined_dataset = concatenate_datasets([slimorca_cleaned, capybara_cleaned])
print(f"\nCombined dataset size: {len(combined_dataset)}")

# Print a sample from the combined dataset
print("\nCombined dataset sample:")
print(combined_dataset[0])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]


Combined dataset size: 2000

Combined dataset sample:
{'conversations': [{'from': 'system', 'value': 'You are an AI assistant. You will be given a task. You must generate a detailed and long answer.', 'weight': None}, {'from': 'human', 'value': 'Write an article based on this "A man has been charged with murder and attempted murder after a woman and the man she was on a date with were stabbed at a restaurant in Sydney, Australia."', 'weight': 0.0}, {'from': 'gpt', 'value': "Title: Tragedy Strikes in Sydney: Victims Stabbed during Dinner Date, Suspect Charged with Murder and Attempted Murder\n\nIntroduction\n\nA seemingly ordinary evening took a tragic turn in Sydney, Australia when a romantic dinner date at a local restaurant ended in bloodshed, as a man allegedly attacked a couple, stabbing both victims. Local authorities have charged the suspect with murder and attempted murder in connection with the horrific crime. In this article, we delve into the chilling incident, the suspect, 

In [5]:
# Load the model and tokenizer
model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"  # Changed to instruct version
model, tokenizer = FastLanguageModel.from_pretrained(model_name, max_seq_length=2048)

# Configure PEFT
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "embed_tokens", "lm_head"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

# Prepare the data collator
response_template = "Assistant: "
collator = DataCollatorForCompletionOnlyLM(tokenizer=tokenizer, response_template=response_template, mlm=False)

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    learning_rate=5e-5,
    fp16=True,
    logging_steps=10,
    max_steps=10,  # Added to limit training time
    output_dir="./finetuned_model_quick",
    report_to="none"
)

# Initialize the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=combined_dataset,
    dataset_text_field="text",
    data_collator=collator,
    args=training_args,
)

# Start fine-tuning
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./finetuned_model_quick")

==((====))==  Unsloth 2024.10.2: Fast Mistral patching. Transformers = 4.46.0.dev0.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.dev925. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:323: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 2
\        /    Total batch size = 4 | Total steps = 10
 "-____-"     Number of trainable parameters = 304,087,040
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: F

Step,Training Loss
10,0.000000


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


In [11]:
import torch
from unsloth import FastLanguageModel

# Prepare the model for inference
FastLanguageModel.for_inference(model)

# Function to generate text
def generate_response(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the model with some prompts
test_prompts = [
    "Human: What is the capital of France?",
    "Human: Explain the concept of machine learning in simple terms.",
    "Human: Write a short poem about autumn.",
]

print("Generated responses:")
for prompt in test_prompts:
    response = generate_response(prompt)
    print(f"\nPrompt: {prompt}")
    print(f"Response: {response}")

Generated responses:

Prompt: Human: What is the capital of France?
Response: Human: What is the capital of France?

AssIst: The capital city of France is Paris. Paris is one of the most famous cities in the world and is known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. It is also home to numerous cafes, boutiques, and art galleries. Paris has a rich history and is a major cultural and tourist destination.

Prompt: Human: Explain the concept of machine learning in simple terms.
Response: Human: Explain the concept of machine learning in simple terms.

Machine learning is a type of artificial intelligence that allows computers to learn and improve from experience without being explicitly programmed. It's like teaching a child to identify different fruits. You show the child an apple and label it as an apple. Then you show the child a banana and label it as a banana. Over time, the child learns to identify different fruits based on thei